# KNN

In [ ]:
import numpy as np
import pandas as pd
import gc
import time
import math
import pickle

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler 

In [ ]:
data = pd.read_pickle('feature_matrix.pickle')
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)
del data
gc.collect();

For KNN it is important to scale features. I try StandardScaler.

In [ ]:
ts = time.time()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.fit_transform(X_test)
time.time()- ts

In [ ]:
ts = time.time()
model = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)
model.fit(X_train, Y_train)
time.time()- ts

In [ ]:
# model = pickle.load(open('knn.pickle', 'rb'))

In [ ]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

sqrt(mean_squared_error(Y_pred, Y_valid))

In [ ]:
pickle.dump(model, open('knn_model.pickle', 'wb'))
pickle.dump(Y_pred, open('knn_train.pickle', 'wb'))
pickle.dump(Y_test, open('knn_test.pickle', 'wb'))

In [ ]:
test = pd.read_csv('test.csv.gz', compression='gzip').set_index('ID')
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('knn_submission.csv', index=False)

LB: ?